In [1]:
import pandas as pd
df = pd.read_csv('../data/hansard_speeches_processed.csv')
df = df.sample(n=100000, random_state = 42)

train = df.sample(frac=0.5,random_state = 42)
test = df.drop(train.index).reset_index(drop=True)
train = train.reset_index(drop=True)

import sys
sys.path.append('../gtm/')
from corpus import GTMCorpus
from gtm import GTM

# Create a GTMCorpus objects (one train and one test set to avoid overfitting the supervised learning algorithm)
train_dataset = GTMCorpus(
    train, # Must contain a column 'doc' with the text of each document and a column 'doc_clean' with the cleaned text of each document.
    labels = "~ party", # The features to predict. Would be "~ gdp" if the df has a column 'gdp'.
    content='~ 1' # To absorb frequent/procedural words
)

test_dataset = GTMCorpus(
    test, # Must contain a column 'doc' with the text of each document and a column 'doc_clean' with the cleaned text of each document.
    labels = "~ party", # The features to predict. Would be "~ gdp" if the df has a column 'gdp'.
    content='~ 1', # To absorb frequent/procedural words,
    vectorizer = train_dataset.vectorizer # pass on the same vectorizer as for the training set (this ensures the document term matrices have the same number of dimensions)
)

# Train the model
tm = GTM(
    train_dataset, 
    test_dataset,
    n_topics = 10,
    doc_topic_prior = 'dirichlet', # other option is "logistic_normal"
    update_prior = False, # no prevalence covariates so no need to update the prior
    predictor_type = 'classifier', # 'regressor' for continuous variables such as GDP
    num_epochs = 10, # No need to run many epochs. I found 10 to work well on 50 000 speeches.
    w_pred_loss = 1, # how much weight should we give to the prediction task in the likelihood?
    print_every = 100, # print progress every x batches
    log_every = 10, # print topic-word dist every x epochs
    batch_size=1024,
)

/home/germain/Documents/topic models/generalized_topic_model/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/germain/Documents/topic models/generalized_topic_model/notebooks/../gtm/corpus.py:77: RuntimeWarning: divide by zero encountered in log
  self.log_word_frequencies = torch.FloatTensor(np.log(np.array(self.M_bow.sum(axis=0)).flatten()))


Epoch   1	Training Loss:2.0650804
Epoch   1	Validation Loss:1.6289113
Epoch   2	Training Loss:0.7424154
Epoch   2	Validation Loss:1.4669273
Epoch   3	Training Loss:0.6949086
Epoch   3	Validation Loss:1.5437074
Epoch   4	Training Loss:0.6834591
Epoch   4	Validation Loss:1.6333249
Epoch   5	Training Loss:0.6769996
Epoch   5	Validation Loss:1.8549132
Epoch   6	Training Loss:0.6572475
Epoch   6	Validation Loss:1.9579830
Epoch   7	Training Loss:0.6448019
Epoch   7	Validation Loss:2.0447390
Epoch   8	Training Loss:0.6509201
Epoch   8	Validation Loss:2.1071834
Epoch   9	Training Loss:0.6435065
Epoch   9	Validation Loss:2.2444357
Epoch  10	Training Loss:0.6502141
Epoch  10	Validation Loss:2.0692913
['amendment', 'hon', 'clause', 'say', 'law', 'debate', 'make', 'member']
['hon', 'year', 'people', 'work', 'say', 'make', 'service', 'time']
['hon', 'school', 'work', 'people', 'year', 'teacher', 'friend', 'education']
['hon', 'year', 'people', 'work', 'make', 'say', 'time', 'friend']
['hon', 'peopl